Defining some configurations for the model

In [ ]:
GPT_CONFIG_124M = {
 "vocab_size": 50257, # Vocabulary size
 "context_length": 1024, # Context length
 "emb_dim": 768, # Embedding dimension
 "n_heads": 12, # Number of attention heads
 "n_layers": 12, # Number of layers
 "drop_rate": 0.1, # Dropout rate
 "qkv_bias": False # Query-Key-Value bias
}

## Basic Model Architecture

In [ ]:
import torch
import torch.nn as nn


class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x




## Tok Embeddings / Pos Embeddings Notes

% Definitions:
% V = vocab size, d = emb dim, B = batch size, T = seq length, L = max context length
% X ∈ {0,…,V-1}^{B×T}, E ∈ ℝ^{V×d}, P ∈ ℝ^{L×d}

$$
\mathrm{tok\_embeds}_{i,t}
= E_{\,X_{i,t},:\,}
\in \mathbb{R}^{V \times d},
\quad
\mathrm{tok\_embeds}\in\mathbb{R}^{B\times T\times d}
$$

$$
\mathrm{pos\_embeds}_{i,t}
= P_{\,t,:\,}
\in \mathbb{R}^{L \times D},
\quad
\mathrm{pos\_embeds}\in\mathbb{R}^{B\times T\times d}
$$

$$
X^{(0)}_{i,t}
= \mathrm{tok\_embeds}_{i,t}
+ \mathrm{pos\_embeds}_{i,t}
\in \mathbb{R}^d
$$

$$
\widetilde X^{(0)}_{i,t}
= \mathrm{Dropout}\bigl(X^{(0)}_{i,t}\bigr)
\in \mathbb{R}^d,
\quad
\widetilde X^{(0)}\in\mathbb{R}^{B\times T\times d}
$$


## Notes on Calls to Embed Functions

$$
\mathrm{tok\_embeds}
= \mathbf{E}\bigl[\mathrm{in\_idx}\bigr]
\quad\Longrightarrow\quad
\bigl(\mathrm{tok\_embeds}\bigr)_{i,t,k}
= E_{\,\mathrm{in\_idx}_{i,t}\,,\,k}
\quad
\begin{aligned}
&\in \mathbb{R},\\
&\mathrm{shape:}\;(B \times T \times d)
\end{aligned}
$$

% 2) Positional lookup:
$$
\mathrm{pos\_indices}
= \bigl[0,1,\dots,T-1\bigr],
\quad
\mathrm{pos\_embeds}
= \mathbf{P}\bigl[\mathrm{pos\_indices}\bigr]
\quad\Longrightarrow\quad
\bigl(\mathrm{pos\_embeds}\bigr)_{i,t,k}
= P_{\,t\,,\,k}
\quad
\begin{aligned}
&\in \mathbb{R},\\
&\mathrm{shape:}\;(B \times T \times d)
\end{aligned}
$$

## Notes on Process up to Logis

```python
x = tok_embeds + pos_embeds    # shape: [B, L, D]
x = self.drop_emb(x)           # still [B, L, D]
x = self.trf_blocks(x)         # [B, L, D] after every block preserves D
x = self.final_norm(x)         # [B, L, D]
```

at this point we have $ X \in \mathbb{R}^{(BS \times SL \times ED)} $

At the logits step, we have  $ \text{logits} \in \mathbb{R}^{(BS \times SL \times VS)} $

## Implement Text

In [ ]:
import tiktoken
import torch
import torch.nn as nn
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [ ]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape:", logits.shape)
print(logits, logits.shape)

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.4476,  0.1726,  0.5354,  ..., -0.3932,  1.5285,  0.8557],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.8422,  0.8243, -0.1098,  ..., -0.1434,  0.2079,  1.2046],
         [ 0.1355,  1.1858, -0.1453,  ...,  0.0869, -0.1590,  0.1552],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>) torch.Size([2, 4, 50257])


In [ ]:
torch.manual_seed(123)
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)
